## Password Manager


In [26]:
#installing the packages
!pip install cryptography

from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64
import os

In [27]:
# Function to derive a key from the master password
def derive_key(master_password, salt):
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(master_password.encode()))
    return key

# Function to write the key to a file (for completeness, though not used here)
def write_key(key):
    with open("key.key", "wb") as key_file:
        key_file.write(key)

# Function to load the key from a file (for completeness, though not used here)
def load_key():
    with open("key.key", "rb") as key_file:
        return key_file.read()

# Function to encrypt the passwords.txt file
def encrypt_file(file_name, key):
    with open(file_name, 'rb') as file:
        data = file.read()
    fernet = Fernet(key)
    encrypted = fernet.encrypt(data)
    with open(file_name, 'wb') as file:
        file.write(encrypted)

# Function to decrypt the passwords.txt file
def decrypt_file(file_name, key):
    with open(file_name, 'rb') as file:
        data = file.read()
    if data:  # Only attempt to decrypt if the file is not empty
        fernet = Fernet(key)
        try:
            decrypted = fernet.decrypt(data)
            with open(file_name, 'wb') as file:
                file.write(decrypted)
        except Exception as e:
            print(f"Decryption failed: {e}")
            return False
    return True

# Load or generate salt
salt_file = 'salt.salt'
if os.path.exists(salt_file):
    with open(salt_file, 'rb') as f:
        salt = f.read()
else:
    salt = os.urandom(16)
    with open(salt_file, 'wb') as f:
        f.write(salt)

master_pwd = input("What is the master password? ")
key = derive_key(master_pwd, salt)

# Ensure the passwords.txt file exists
if not os.path.exists('passwords.txt'):
    open('passwords.txt', 'w').close()

# Decrypt the passwords.txt file at the start, if possible
if decrypt_file('passwords.txt', key):
    print("Decryption successful")
else:
    print("Decryption failed")

fer = Fernet(key)

# Function to view stored passwords
def view():
    with open('passwords.txt', 'r') as f:
        for line in f.readlines():
            data = line.rstrip()
            if "|" in data:  # Check if the delimiter is present
                user, passw = data.split("|")
                print("User:", user, "| Password:",
                      fer.decrypt(passw.encode()).decode())
            else:
                print(f"Invalid line format: {data}") # Inform the user about the invalid line

# Function to add a new account and password
def add():
    name = input('Account Name: ')
    pwd = input('Password: ')
    encrypted_pwd = fer.encrypt(pwd.encode()).decode()
    with open('passwords.txt', 'a') as f:
        f.write(name + "|" + encrypted_pwd + '\n')

# Function to change the master password
def change_master_password():
    global key, fer
    new_master_pwd = input("Enter the new master password: ")
    new_key = derive_key(new_master_pwd, salt)
    encrypt_file('passwords.txt', new_key)
    key = new_key
    fer = Fernet(key)
    print("Master password changed successfully.")

while True:
    mode = input("Would you like to add a password, view existing ones, or change the master password? (add/view/change) or press q to quit: ").lower()

    if mode == "q":
        # Encrypt the passwords.txt file before quitting
        encrypt_file('passwords.txt', key)
        print("Bye")
        break
    elif mode == "view":
        view()
    elif mode == "add":
        add()
    elif mode == "change":
        change_master_password()
    else:
        print("Invalid mode")
        continue

# Encrypt the passwords.txt file before quitting
encrypt_file('passwords.txt', key)


What is the master password? san
Decryption successful
Would you like to add a password, view existing ones, or change the master password? (add/view/change) or press q to quit: add
Account Name: san12
Password: 12345678
Would you like to add a password, view existing ones, or change the master password? (add/view/change) or press q to quit: add
Account Name: real12
Password: 2345www
Would you like to add a password, view existing ones, or change the master password? (add/view/change) or press q to quit: view
User: san12 | Password: 12345678
User: real12 | Password: 2345www
Would you like to add a password, view existing ones, or change the master password? (add/view/change) or press q to quit: q
Bye
